In [1]:
import pandas as pd
from pprint import pprint
import glob
import pandas as pd
import random
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from config import event_types
random.seed(42)
pd.set_option('display.max_rows', None)
# jupyter nbconvert --to script 04b_combine_results_find_disagreement.ipynb
# sudo kill -9 $(nvidia-smi | awk 'NR>8 {print $5}' | grep -E '^[0-9]+$')
version = 2
from config import event_types

def prepare_df(df, type="biolord"):
    global event_types
    df = df.copy()
    print("Loaded file len: ",len(df))
    df.columns = ['Document' if i=='DOCUMENT' else i for i in df.columns]
    df['Sent_ID'] = df['Events'].apply(lambda x: [f"{i:04d}" for i in range(len(x))])
    df = df.explode(["Sent_ID","Events"])
    print("After exploding file len: ",len(df))
    df['UID'] = df['SUBJECT_ID'].astype(str) + "_" + df['ROW_ID'].astype(str) + "_" + df['Sent_ID'].astype(str)
    df = df.dropna(subset="Events")
    df['Event_Name'] = df['Events'].apply(lambda x: x['event'])
    df['Sentence'] = df['Events'].apply(lambda x: x['text'])
    df['Time'] = df['Events'].apply(lambda x: x['event_detection_time'])
    
    if type == "dictionary":
        df['Keyword'] = df['Events'].apply(lambda x: x['keyword'])
        df['Lemma'] = df['Events'].apply(lambda x: x['lemma'])
        df['Keyword_Position'] = df['Events'].apply(lambda x: x['keyword_position'])
        
    if type == "biolord":
        df['Similarity'] = df['Events'].apply(lambda x: x['similarity'])
        df['Similarity'] = df['Similarity'].apply(lambda x: {k:v for (k,v) in x.items() if k!="Alert And Oriented"})
        for ET in event_types:
            df[f"{ET}_similarity"] = df['Similarity'].apply(lambda x:x[ET])
            
    return df
df_dictionary = prepare_df(pd.read_pickle(f"../exports/03b_selected_reports_with_event_log_only_dictionary_v{version}_orig/combined.pkl"),type="dictionary")
df_dictionary_wc = prepare_df(pd.read_pickle(f"../exports/03b_selected_reports_with_event_log_only_dictionary_v{version}_withcontext/combined.pkl"),type="dictionary")

# df_biolord = prepare_df(pd.read_pickle(f"../exports/selected_reports_with_event_log_only_biolord_v{version}/combined.pkl"), type = "biolord")
df_dictionary.to_pickle("../exports/04b_dictionary_features.pkl")
df_dictionary_wc.to_pickle("../exports/04b_dictionary_features_withcontext.pkl")





Loaded file len:  12524
After exploding file len:  280066
Loaded file len:  12524
After exploding file len:  280066


In [2]:
df_dictionary = pd.read_pickle("../exports/04b_dictionary_features.pkl")
df_dictionary_wc = pd.read_pickle("../exports/04b_dictionary_features_withcontext.pkl")


In [3]:
N1 = 10
def combine_lists(x):
    combined = []
    for item in x:
        if item:
            if isinstance(item, (list, tuple)):
                combined.extend(item)
            else:
                combined.append(item)
    combined = [i for i in combined if i]
    res = set(combined)
    return list(res)
from config import event_types
import os
os.makedirs("../exports/04c_groundtruth/P-SET/Generated", exist_ok=True)
event_types = ['Sleep']
for ET in event_types:
    df_dictionary[f'Event_Name_{ET}'] = df_dictionary['Event_Name'].apply(lambda x: ET in x)
    df_dictionary_wc[f'Event_Name_{ET}'] = df_dictionary_wc['Event_Name'].apply(lambda x: f"{ET}" in x)
    df_dictionary_wc[f'Event_Name_{ET}context'] = df_dictionary_wc['Event_Name'].apply(lambda x: f"{ET}context" in x)
    row_id_to_doc_ET = df_dictionary.groupby('ROW_ID')[f'Event_Name_{ET}'].max().to_dict()
    df_dictionary[f'Event_Name_{ET}_doc'] = df_dictionary['ROW_ID'].apply(lambda x: row_id_to_doc_ET[x])
    people_ranked_by_ET = df_dictionary.groupby('SUBJECT_ID')[f'Event_Name_{ET}'].sum().sort_values(ascending=False).index.tolist()
    df_dictionary_topN1 = df_dictionary[df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N1])]
    sentences_w_et_topn1 = df_dictionary_topN1[df_dictionary_topN1[f'Event_Name_{ET}']]
    df_dictionary_rest = df_dictionary[~df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N1])]
    sentences_w_et_rest = df_dictionary_rest[df_dictionary_rest[f'Event_Name_{ET}']].sample(len(sentences_w_et_topn1), random_state=42)
    print(len(sentences_w_et_topn1), len(sentences_w_et_rest))
    Sentences_with_ET = pd.concat([sentences_w_et_topn1, sentences_w_et_rest]).reset_index(drop=True)
    Sentences_with_ET['is_keyword_present'] = True
    df_dictionary_rest2 = df_dictionary[~df_dictionary.UID.isin(Sentences_with_ET.UID)]
    sentences_w_et_rest2 = df_dictionary_rest2[df_dictionary_rest2[f'Event_Name_{ET}']].sample(len(sentences_w_et_topn1), random_state=42)
    Sentences_with_ET = pd.concat([Sentences_with_ET, sentences_w_et_rest2]).reset_index(drop=True)
    Sentences_with_ET['is_keyword_present'] = True
    print("Sentences from top N1: ",len(sentences_w_et_topn1), 
          "Sentences from rest, with ET: ",len(sentences_w_et_rest), 
          "Sentences from rest, with ET: ",len(sentences_w_et_rest2))

    sentences_wo_et_w_context = df_dictionary_wc[(df_dictionary_wc[f'Event_Name_{ET}']==False)&(df_dictionary_wc[f'Event_Name_{ET}context']==True)]
    sentences_wo_et_w_context_sample = sentences_wo_et_w_context.sample(2*len(sentences_w_et_topn1), random_state=42)
    sentences_wo_et_topn1 = (df_dictionary_topN1[df_dictionary_topN1[f'Event_Name_{ET}']==False]
                            .sample(2*len(sentences_w_et_topn1), random_state=42))
    sentences_wo_et_topn1_sample = sentences_wo_et_topn1[~sentences_wo_et_topn1.UID.isin(sentences_wo_et_w_context.UID)].iloc[:len(sentences_w_et_topn1)]
    Sentences_without_ET = pd.concat([sentences_wo_et_w_context_sample, sentences_wo_et_topn1_sample]).reset_index(drop=True)
    Sentences_without_ET['is_keyword_present'] = False
    print("Sentences without et but with et context: ",len(sentences_wo_et_w_context_sample), 
          "Sentences without et: ",len(sentences_wo_et_topn1_sample))
    Sentences = pd.concat([Sentences_with_ET, Sentences_without_ET]).reset_index(drop=True)
    
    Sentences[f'Sent_gt_{ET}'] = None
    Sentences['negation'] = False
    Sentences['good_example'] = False
    Sentences["comment"] = ""
    
    Documents_with_ET = df_dictionary_topN1[df_dictionary_topN1[f'Event_Name_{ET}_doc']]
    Documents_with_ET['is_keyword_present'] = True
    Documents_without_ET =  df_dictionary_topN1[~df_dictionary_topN1[f'Event_Name_{ET}_doc']]
    Documents_without_ET['is_keyword_present'] = False
    Documents = pd.concat([Documents_with_ET, Documents_without_ET]).reset_index(drop=True)
    Documents['Document'] = Documents['Document'].apply(lambda x: x.lower())
    Documents = Documents.groupby(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'AGE', 'LOS_DAYS', 'IS_ALIVE', 'DURATION_NOTE'])[["Event_Name","Keyword","Document","Lemma"]].agg(lambda x:combine_lists(x)).reset_index()
    Documents['Document'] = [i[0] for i in Documents['Document']]
    print(f"For event type {ET}, N1={N1}, Sentences_with_ET={len(Sentences_with_ET)}, Sentences_without_ET={len(Sentences_without_ET)}, Sentences={len(Sentences)},Documents_with_ET={Documents_with_ET.ROW_ID.nunique()}, Documents_without_ET={Documents_without_ET.ROW_ID.nunique()}, Documents={Documents.ROW_ID.nunique()}")
    
    if os.path.exists(f"../exports/04b_groundtruth/P-SET/Annotated/{ET}_Sentences.pkl"):
        old_annotations_sent = pd.read_pickle(f"../exports/04b_groundtruth/P-SET/Annotated/{ET}_Sentences.pkl")
        uid_to_gt = old_annotations_sent.groupby(['UID'])[f'Sent_gt_{ET}'].max().to_dict()
        uid_to_negation = old_annotations_sent.groupby(['UID'])[f'negation'].max().to_dict()
        uid_to_good_example = old_annotations_sent.groupby(['UID'])[f'good_example'].max().to_dict()
        uid_to_comment = old_annotations_sent.groupby(['UID'])[f'comment'].max().to_dict()
        Sentences[f'Sent_gt_{ET}'] = Sentences.UID.map(lambda x: uid_to_gt.get(x, None))
        Sentences['negation'] = Sentences.UID.map(lambda x: uid_to_negation.get(x, False))
        Sentences['good_example'] = Sentences.UID.map(lambda x: uid_to_good_example.get(x, False))
        Sentences['comment'] = Sentences.UID.map(lambda x: uid_to_comment.get(x, ""))
        print(len(Sentences), "previously annotated:", len(Sentences[Sentences.UID.isin(old_annotations_sent.UID)]))
        
    if os.path.exists(f"../exports/04b_groundtruth/P-SET/Annotated/{ET}_Documents.pkl"):
        old_annotations_doc = pd.read_pickle(f"../exports/04b_groundtruth/P-SET/Annotated/{ET}_Documents.pkl")
        row_id_to_gt = old_annotations_doc.groupby(['ROW_ID'])[f'Doc_gt_{ET}'].max().to_dict()
        row_id_to_negation = old_annotations_doc.groupby(['ROW_ID'])[f'negation'].max().to_dict()
        row_id_to_good_example = old_annotations_doc.groupby(['ROW_ID'])[f'good_example'].max().to_dict()
        row_id_to_comment = old_annotations_doc.groupby(['ROW_ID'])[f'comment'].max().to_dict()
        Documents[f'Doc_gt_{ET}'] = Documents.ROW_ID.map(lambda x: row_id_to_gt.get(x,None))
        Documents['negation'] = Documents.ROW_ID.map(lambda x: row_id_to_negation.get(x,False))
        Documents['good_example'] = Documents.ROW_ID.map(lambda x: row_id_to_good_example.get(x,False))
        Documents['comment'] = Documents.ROW_ID.map(lambda x: row_id_to_comment.get(x,""))
        print(len(Documents), len(Documents[Documents.ROW_ID.isin(old_annotations_doc.ROW_ID)]))
        
    if ET == "Sleep":
        print(Sentences.Sent_gt_Sleep.value_counts(dropna=False), Documents.Doc_gt_Sleep.value_counts(dropna=False))
    Sentences.to_pickle(f"../exports/04c_groundtruth/P-SET/Generated/{ET}_Sentences.pkl")
    Documents.to_pickle(f"../exports/04c_groundtruth/P-SET/Generated/{ET}_Documents.pkl")

150 150
Sentences from top N1:  150 Sentences from rest, with ET:  150 Sentences from rest, with ET:  150
Sentences without et but with et context:  300 Sentences without et:  150
For event type Sleep, N1=10, Sentences_with_ET=450, Sentences_without_ET=450, Sentences=900,Documents_with_ET=83, Documents_without_ET=119, Documents=202
900 previously annotated: 450
202 202
Sent_gt_Sleep
None     450
True     251
False    191
NaN        8
Name: count, dtype: int64 Doc_gt_Sleep
False    113
True      77
NaN       12
Name: count, dtype: int64


/tmp/ipykernel_3687631/1721388753.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_3687631/1721388753.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_without_ET['is_keyword_present'] = False


In [4]:
sentences_wo_et_topn1_sample[sentences_wo_et_topn1_sample.UID.isin(sentences_wo_et_w_context.UID)][['UID','Sentence']]

,UID,Sentence


In [ ]:
old_annotations_sent.head(1), Sentences.head(1)
# sent_to_negation.values()

In [ ]:
ET='Sleep'
os.path.exists(f"../exports/04_groundtruth/P-SET/Annotated/{ET}_Documents.pkl")

In [ ]:
old_annotations.columns

In [ ]:
ET = "Sleep"
old_annotations = pd.read_pickle(f"../exports/04_groundtruth/P-SET/Annotated/{ET}_Documents.pkl")
new_annotations = pd.read_pickle(f"../exports/04b_groundtruth/P-SET/Generated/{ET}_Documents.pkl")

sent_to_gt = old_annotations.groupby('Document')[f'Doc_gt_{ET}'].max().to_dict()
len([i for i in new_annotations.Document.tolist() if i in old_annotations.Document.tolist()])

In [ ]:
new_annotations.Sentence.tolist()

In [ ]:
len(df_dictionary_topN2[~df_dictionary_topN2[f'Event_Name_{ET}']]), len(Sentences_with_ET)

In [ ]:
Sentences.columns

In [ ]:
Documents.Document.iloc[0]

In [ ]:
# N = 10
# from config import event_types
# import os
# os.makedirs("../exports/04_groundtruth/P-SET/Generated", exist_ok=True)
# for ET in event_types:
#     top_ET = df_dictionary_exploded[df_dictionary_exploded.Event_Name==ET][["SUBJECT_ID","Event_Name"]].value_counts().reset_index().iloc[:N]
#     selected_patient_sentences = df_dictionary[df_dictionary.SUBJECT_ID.isin(top_ET['SUBJECT_ID'].tolist())]
#     total_ET = top_ET[top_ET.Event_Name==ET]['count'].sum()
#     num_sentences = len(selected_patient_sentences.UID.unique())
#     num_reports = len(selected_patient_sentences.ROW_ID.unique())
#     num_patients = len(selected_patient_sentences.SUBJECT_ID.unique())
#     print(f"***{ET}***",
#           f"{ET} counts:{total_ET}\n",
#         # f"Event counts:\n{selected_patient_sentences.Event_Name.value_counts()}\n"
#         f"Unique Sentences: {num_sentences}\n",
#         f"Unique Reports: {num_reports}\n",
#         f"Unique Patients: {num_patients}\n"
#     )
#     selected_patient_sentences.to_pickle(f"../exports/04_groundtruth/P-SET/Generated/04_{ET}_{num_patients}_{num_reports}_{num_sentences}_{total_ET}.pkl")

In [ ]:
import pandas as pd
df = pd.read_pickle(f"../exports/04_dictionary_features.pkl")
ET = 'Sleep'
df_sentence_with_sleep = (df[df.Event_Name.apply(lambda x: ET in x)])
df_sentence_without_sleep = (df[df.Event_Name.apply(lambda x: ET not in x)])
top_10_patients = df_sentence_with_sleep.SUBJECT_ID.value_counts().head(10).index.tolist()
all_patients = pd.read_pickle("../exports/02b_filtered_patient_reports_7_14_days.pkl")['SUBJECT_ID'].unique().tolist()
len(top_10_patients), len(set(top_10_patients).intersection(set(all_patients)))
# df_top_10_patient_sentences_with_sleep = df_sentence_with_sleep[df_sentence_with_sleep.SUBJECT_ID.isin(top_10_patients)]
# df_top_10_patient_sentences_without_sleep = df_sentence_without_sleep[df_sentence_without_sleep.SUBJECT_ID.isin(top_10_patients)]
# df_not_top_10_patient_sentences = df_sentence_with_sleep[~df_sentence_with_sleep.SUBJECT_ID.isin(top_10_patients)]
# print("top 10 patient sentences:", len(df_top_10_patient_sentences_with_sleep))
# print("top 10 patients sentences without sleep:", len(df_top_10_patient_sentences_without_sleep))
# print("other patient sentences:", len(df_not_top_10_patient_sentences))

In [ ]:
df_top_10_patient_sentences_without_sleep.SUBJECT_ID.value_counts()

In [ ]:
docid_isevent = df[df.SUBJECT_ID.isin(top_10_patients)].groupby(['ROW_ID','SUBJECT_ID'])['Event_Name'].agg(lambda x: any([ET in j for j in x])).reset_index()
docid_isevent[docid_isevent.Event_Name==True]['SUBJECT_ID'].value_counts()
res = docid_isevent.groupby(["Event_Name", "SUBJECT_ID"]).size().unstack(fill_value=0)
res, res.sum(axis=1)

In [ ]:
sentid_isevent = df[df.SUBJECT_ID.isin(top_10_patients)].groupby(['UID','SUBJECT_ID'])['Event_Name'].agg(lambda x: any([ET in j for j in x])).reset_index()
sentid_isevent[sentid_isevent.Event_Name==True]['SUBJECT_ID'].value_counts()
res = sentid_isevent.groupby(["Event_Name", "SUBJECT_ID"]).size().unstack(fill_value=0)
res, res.sum(axis=1)

In [ ]:
for N2 in range(7):
    res = df_sentence_without_sleep[df_sentence_without_sleep.SUBJECT_ID.isin(top_10_patients[:N2])]
    print(N2,len(res))
    
2366 + 2502, 121 + 81

In [ ]:
df[df.SUBJECT_ID.isin(top_10_patients)].ROW_ID.nunique()

In [ ]:
2355 + 147

df.LOS